# Coupled-cluster doubles

In this notebook we set up a general solver for CCD given matrices $h_{p}^{q}$ and $u^{pq}_{rs}$, where the latter is _anti-symmetric_.

In [10]:
import pickle
import numpy as np

We start by retrieving the pickled one-body and two-body matrix elements generated in the `matrix_elements.ipynb`-notebook.

In [15]:
with open("dat/h.pkl", "rb") as f:
    h = pickle.load(f)
with open("dat/u.pkl", "rb") as f:
    u = pickle.load(f)

## Iteration scheme

Using the equations (A7a and A7b) listed in "Ab initio quantum dynamics using coupled-cluster" by Simen Kvaal we can define iterations to find the amplitudes $\tau$. We define (note that there are no summing in these expressions)

\begin{align}
    f_{ij}^{ab} &= \frac{\partial}{\partial \lambda_{ab}^{ij}}\mathcal{E}_{H^{(1)}} \\
        &= -h_{c}^{a}\tau_{ij}^{bc}P(ab) + h_{i}^{k}\tau_{jk}^{ab}P(ij),
\end{align}

where $P(ij)$ is the anti-symmetrization operator permuting the two indices $i$, $j$ and subtracting the result. We start by assuming that $h$ is a diagonal matrix thus making a simplified expression for the right hand side of $f$.

\begin{align}
    f_{ij}^{ab} &= -h_{a}^{a}\tau_{ij}^{ba}P(ab) + h_{i}^{i}\tau_{ji}^{ab}P(ij) \\
        &= -h_{a}^{a}\tau_{ij}^{ba} + h_{b}^{b}\tau_{ij}^{ab}
            + h_{i}^{i}\tau_{ji}^{ab} - h_{j}^{j}\tau_{ij}^{ab} \\
        &= (h_{a}^{a} + h_{b}^{b} - h_{i}^{i} - h_{j}^{j})\tau_{ij}^{ab} \\
        &= -D_{ij}^{ab}\tau_{ij}^{ab},
\end{align}

where $D = D(h)$.
The equations from A7b in Kvaal's article is given by

\begin{align}
    g_{ij}^{ab} &= \frac{\partial}{\partial\lambda_{ab}^{ij}}\mathcal{E}_{H^{(2)}} \\
        &= - \frac{\tau^{dc}_{jm} \tau^{ba}_{in}}{2} u^{mn}_{dc} P(ij)
            + \frac{\tau^{dc}_{ji} \tau^{ba}_{mn}}{4} u^{mn}_{dc}
            + \frac{\tau^{dc}_{ji} u^{ba}_{dc}}{2} \\
            &\qquad
            - \tau^{ac}_{im} \tau^{bd}_{jn} u^{mn}_{dc} P(ij)
            + \tau^{ac}_{im} u^{bm}_{jc} P(ab) P(ij)
            - \frac{\tau^{ac}_{ji} \tau^{bd}_{mn}}{2} u^{mn}_{dc} P(ab) \\
            &\qquad
            - \tau^{ac}_{ji} u^{bm}_{cm} P(ab)
            + \frac{\tau^{ba}_{mn} u^{mn}_{ji}}{2}
            + \tau^{ba}_{im} u^{mn}_{jn} P(ij) + u^{ba}_{ji}.
\end{align}

We define $g = g(u, \tau)$ and $f = f(h, \tau)$. We have that
\begin{align}
    f + g = 0 \implies -f = g.
\end{align}

We now define an iteration by introducing

\begin{align}
    \tau^{(k + 1)} = \frac{g(u, \tau^{(k)})}{D},
\end{align}

and start with the initial guess

\begin{align}
    \tau^{(0)} = \frac{u}{D}.
\end{align}

In [19]:
def compute_amplitudes(h, u, diagonal_h=True):
    if not diagonal_h:
        raise NotImplementedError("This function currently only supports a diagonal h-matrix")

In [20]:
compute_amplitudes(1, 2, False)

NotImplementedError: This function currently only supports a diagonal h-matrix